In [16]:
n = 5
G = graphs.CompleteGraph(n)
edgelist = graphs.CompleteGraph(n).edges(labels=False)
matroid = Matroid(graph=edgelist, groundset=edgelist)
group = SymmetricGroup(range(n))
num_conjugacy_classes = len(group.conjugacy_classes())

def action_on_flats(g, m):
    def action_on_groundset(g, x):
        return tuple(sorted(g(y) for y in x))
    
    return tuple(sorted([action_on_groundset(g,x) for x in m]))

def action_on_fymonomials(g, monomial):
    return tuple(sorted([action_on_flats(g,m) for m in monomial]))

def stab(G, m, action):
    return G.subgroup(set(g for g in G if action(g, m) == tuple(m)))

def orbits(G, X, action):
    def orbit(G, m):
        return set(sorted(action(g, m) for g in G))
    return sorted(set(tuple(orbit(G, x)) for x in X))

def perm_representation(G, X, action):
    fn = ClassFunction(G, [0] * num_conjugacy_classes)
    for orbit in orbits(G, X, action):
        stabilizer_group = stab(G, orbit[0], action)
        chi = stabilizer_group.trivial_character()
        fn += chi.induct(G)

    return fn

In [17]:
flats = [list(matroid.flats(i)) for i in range(n)][1:]

for i in range(len(flats)):
    print(i)
    for x, y in perm_representation(group, flats[i], action_on_flats).decompose():
        print(x, y.values())

In [18]:
def generate_weights(rank):
    weights = set() 
    for i in range(1, rank):
        for j in range(rank):
            weight = [0] * rank
            
            if i >= j:
                weight[i] = j
                weights.add(tuple(weight))

            if rank - (i+1) > 1:
                y = generate_weights(rank - (i + 1))  # Recursion to get the complete list of weights...
                for x in y:
                    temp_weight = weight.copy()
                    weights.add(tuple(temp_weight[:i+1] + x)) 

    return [list(w) for w in weights]

rank = n - 1
weights = generate_weights(rank)

In [19]:
fy_monomials_list = [[] for _ in range(rank)]

def generate_monomials(weight, flats):
    if sum(weight) == 0: return [[]]
    # Find the first non-zero weight
    start_index = next(i for i in range(len(weight)) if weight[i] != 0)
    result = []

    for initial_flat in flats[start_index]:
        initial_monomial = [initial_flat] * weight[start_index]
        potential_combinations = [initial_monomial]

        for i in range(start_index + 1, len(weight)):
            if weight[i] != 0:
                new_combinations = []
                for flat in flats[i]:
                    for combo in potential_combinations:
                        if combo[-1].issubset(flat):
                            new_combo = combo + [flat] * weight[i]
                            new_combinations.append(new_combo)
                potential_combinations = new_combinations

        result.extend(potential_combinations)

    return result

for weight in weights:
    fy_monomials_list[sum(weight)].extend(generate_monomials(weight, flats))

[len(x) for x in fy_monomials_list]

[1, 41, 41, 1]

In [20]:
for i in range(n-1):
    print("rank: " + str(i))
    for x, y in perm_representation(group, fy_monomials_list[i], action_on_fymonomials).decompose():
        print(x, y.values())

rank: 0
1 [1, 1, 1, 1, 1, 1, 1]
rank: 1
5 [1, 1, 1, 1, 1, 1, 1]
4 [4, 2, 0, 1, -1, 0, -1]
1 [5, -1, 1, -1, -1, 1, 0]
3 [5, 1, 1, -1, 1, -1, 0]
rank: 2
5 [1, 1, 1, 1, 1, 1, 1]
4 [4, 2, 0, 1, -1, 0, -1]
1 [5, -1, 1, -1, -1, 1, 0]
3 [5, 1, 1, -1, 1, -1, 0]
rank: 3
1 [1, 1, 1, 1, 1, 1, 1]
